<br>

# Introdução

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
from paths import *

<br>

# Plotly

In [ ]:
df = pd.read_csv(
    #os.path.join('data', 'tabs', 'tab_municipio_allinfos.csv'),
    'https://raw.githubusercontent.com/michelmetran/pl251/main/data/tabs/tab_municipio_allinfos.csv',
)
df

In [ ]:
# Classes
#class_1 = 'nome_ugrhi'
class_1 = 'nome_rm'
class_2 = 'unidade'

# Listas para Iteração
list_class_1 = list(set(df[class_1]))
list_class_1.sort()
list_class_2 = list(set(df[class_2]))
list_class_2.sort()

list_class = []
for i in list_class_1:
    #print('Para a {} "{}" temos:'.format(class_1, i))
    df_temp = df[df[class_1] == i].copy()
    list_subclass = []
    for j in list_class_2:
        #print('Na {} "{}"'.format(class_2, j))
        df_temp2 = df_temp[df_temp[class_2] == j].copy()
        list_mun = list(df_temp2['municipio_nome'])
        if len(list_mun) == 0:
            list_text = ''
            list_subclass.append(list_text)
        elif len(list_mun) != 0:
            list_mun.sort()
            list_text = '<br>'.join(list_mun)
            #print('> {}'.format(list_text))
            list_subclass.append(list_text)
    list_class.append(list_subclass)

In [ ]:
# Convert to Array
df_array = df.groupby(by=[class_1])['unidade'].value_counts().sort_index()
df_array = df_array.unstack()
#print(df_array)
#display(df_array)

# Principal Data
data_1 = df_array.replace(np.nan, 0, regex=True).to_numpy()
data_1 = data_1.astype(np.int64)
data_2 = np.array([int(i) for i in list(df_array.sum(axis=1))])
data_2 = np.reshape(data_2, (1, len(df_array))).T
data_3 = np.array([int(i) for i in list(df_array.sum(axis=0))])
data_3 = np.reshape(data_3, (1, len(df_array.columns)))
data_4 = np.array([[data_3.sum()]])

# Labels
x_label = list(df_array.columns)
y_label = list(df_array.index)

# Invert Matrices
data_1 = data_1[::-1]
data_2 = data_2[::-1]
data_3 = data_3[::-1]
list_class = list_class[::-1]
#x_label = x_label[::-1]
y_label = y_label[::-1]

# Results
print(data_1)
print(data_2)
print(data_3)
print(data_4)
print(y_label)

In [ ]:
y_label = [(x.replace('Região Metropolitana do ', 'RM<br>').
  replace('Região Metropolitana de ', 'RM<br>').
  replace('Região Metropolitana da ', 'RM<br>').
  replace('Aglomeração Urbana de ', 'AU<br>').
  replace('-AU- Piracicaba', '').
  replace('Vale do Paraíba e Litoral Norte', 'V. Paraíba e L. Norte')
 ) for x in y_label]

y_label

In [ ]:
# Create Subplots
fig = make_subplots(
    rows=2,
    cols=2,
    row_heights=[0.85, 0.1],
    column_widths=[0.85, 0.1],
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
    print_grid=True,    
    shared_xaxes=True,
    shared_yaxes=True,    
)

colorscale = 'YlGnBu'
# Make Annotated Heatmap
fig1 = ff.create_annotated_heatmap(
    data_1,
    text=list_class,
    x=[i.replace(' - ', '<br>') for i in x_label],
    y=y_label,
    colorscale=colorscale,
    #font_colors=['black'],
    hoverinfo='text',
)
fig2 = ff.create_annotated_heatmap(
    data_2,
    text=[['Nº Mun.<br>{}'.format(i.replace('<br>', ' ')) for i in y_label]],
    colorscale=colorscale,
    #font_colors=['black'],
    hoverinfo='text',
)
fig3 = ff.create_annotated_heatmap(
    data_3,
    text=[['Nº Mun.<br>{}'.format(i) for i in x_label]],
    colorscale=colorscale,
    #font_colors=['black'],
    hoverinfo='text',
)
fig4 = ff.create_annotated_heatmap(
    data_4,
    text=[['Nº Mun. Estado']],
    colorscale=colorscale,
    #font_colors=['black'],
    hoverinfo='text',
)

# Adds
fig.add_trace(fig1.data[0], 1, 1)
fig.add_trace(fig2.data[0], 1, 2)
fig.add_trace(fig3.data[0], 2, 1)
fig.add_trace(fig4.data[0], 2, 2)

# Label
annot1 = list(fig1.layout.annotations)
annot2 = list(fig2.layout.annotations)
annot3 = list(fig3.layout.annotations)
annot4 = list(fig4.layout.annotations)
for k in range(len(annot2)):
    annot2[k]['xref'] = 'x2'
    annot2[k]['yref'] = 'y2'
for k in range(len(annot3)):
    annot3[k]['xref'] = 'x3'
    annot3[k]['yref'] = 'y3'
for k in range(len(annot4)):
    annot4[k]['xref'] = 'x4'
    annot4[k]['yref'] = 'y4'
fig.update_layout(annotations=annot1+annot2+annot3+annot4)

# Updates
fig.update_layout(
    #title_text='URAEs <i>vs.</i> RMs/AUs',
    width=700,
    height=700,
    showlegend=False,
    font_size=11,
    hovermode='closest',
    #hovermode='x',
    xaxis1_showticklabels=True,
    yaxis1_showticklabels=True,
    xaxis2_showticklabels=False,
    yaxis2_showticklabels=False,
    xaxis3_showticklabels=False,
    yaxis3_showticklabels=False,
    xaxis4_showticklabels=False,
    yaxis4_showticklabels=False,
    xaxis=go.layout.XAxis(
        tickangle=-40,
        side='top',        
    ),    
    dragmode=False,
)
fig.update_traces(
    #colorbar_tickangle=60,
    #selector=dict(type='heatmap')
)
fig.update_layout(
    xaxis_domain=[0.0, 0.85],
    yaxis_domain=[0.15, 1.0],
)

# Results
config = {
    'displayModeBar': False,
    #'scrollZoom': False,
    'displaylogo': False,
    #'staticPlot': True,
}
fig.show(config=config)
fig.write_html(os.path.join(imgs_path, 'matrix_{}.html'.format(class_1.replace('nome_', ''))))